In [ ]:
! pip install git+https://github.com/IBM/tsfm.git
! pip install transformers

In [ ]:
import pandas as pd
import torch
from transformers import PatchTSTForPrediction

# Load custom CSV file
data_path = "/kaggle/input/iotbpm/augmented_data.csv"
data = pd.read_csv(data_path)

# Assuming 'Geometry-Dash' and 'Clash-Royale' are column names
geometry_dash_values = data['Geometry-Dash'].values
clash_royale_values = data['Clash-Royale'].values

# Pad or truncate the sequences to match the model's input length (512)
max_sequence_length = 512
geometry_dash_values = torch.tensor(geometry_dash_values[:max_sequence_length], dtype=torch.float32)
clash_royale_values = torch.tensor(clash_royale_values[:max_sequence_length], dtype=torch.float32)

# Concatenate the values of both columns into a single tensor
input_values = torch.stack([geometry_dash_values, clash_royale_values], dim=1) # Shape: [sequence_length, num_channels]

model = PatchTSTForPrediction.from_pretrained("chungimungi/PatchTST-2-input-channels")

# Ensure the model is in evaluation mode
model.eval()

# During inference, one only provides past values, the model outputs future values
with torch.no_grad():
  outputs = model(past_values=input_values.unsqueeze(0)) # Add batch dimension
  prediction_outputs = outputs.prediction_outputs

# Extract predicted future values
predicted_values = prediction_outputs.squeeze(0) # Remove batch dimension

In [ ]:
import matplotlib.pyplot as plt

# Use the values from the code you provided
geometry_dash_values = geometry_dash_values.numpy()
clash_royale_values = clash_royale_values.numpy()
predicted_geometry_dash = predicted_values[:, 0].numpy()
predicted_clash_royale = predicted_values[:, 1].numpy()

# Create a figure and axis for Geometry-Dash
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 20))

# Plot Geometry-Dash
ax1.plot(geometry_dash_values, label='Actual')
ax1.plot(predicted_geometry_dash, label='Predicted')
ax1.set_xlabel('Time')
ax1.set_ylabel('Geometry-Dash Value')
ax1.set_title('Geometry-Dash: Actual vs Predicted')
ax1.legend()

# Create a figure and axis for Clash-Royale
ax2.plot(clash_royale_values, label='Actual')
ax2.plot(predicted_clash_royale, label='Predicted')
ax2.set_xlabel('Time')
ax2.set_ylabel('Clash-Royale Value')
ax2.set_title('Clash-Royale: Actual vs Predicted')
ax2.legend()

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Show the plot
plt.show()